### Comparing Models and Vectorization Strategies for Text Classification

This try-it focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in scikitlearn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.

**Note:** The original dataset contains 200K rows of data. It is best to try to use the full dtaset. If the original dataset is too large for your computer, please use the 'dataset-minimal.csv', which has been reduced to 100K.

In [4]:
import pandas as pd

In [6]:
df = pd.read_csv('../text_data/dataset.csv')

In [8]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [11]:
pd.DataFrame({'model': ['Logistic', 'Decision Tree', 'Bayes'], 
             'best_params': ['', '', ''],
             'best_score': ['', '', '']}).set_index('model')

,best_params,best_score
model,,
Logistic,,
Decision Tree,,
Bayes,,


### Setting Up the Environment and Importing Necessary Libraries

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import FunctionTransformer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import nltk
import time

# Download NLTK data if needed
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mitalibansal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Preprocessing

In [16]:
# Text preprocessing functions
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def stemming(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

def lemmatizing(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Prepare a FunctionTransformer for both stemming and lemmatizing
stem_transformer = FunctionTransformer(lambda x: x.apply(stemming))
lemmatize_transformer = FunctionTransformer(lambda x: x.apply(lemmatizing))

# Prepare vectorizers
count_vectorizer = CountVectorizer(stop_words='english', max_features=5000)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

### Train / Test Split

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X = df['text']
y = df['humor']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

### Creating Pipelines and Grid Search

In [22]:
# Prepare classifiers
logistic = LogisticRegression(max_iter=1000)
decision_tree = DecisionTreeClassifier()
bayes = MultinomialNB()

# Define pipelines
pipelines = [
    ('logistic_stem_count', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', count_vectorizer),
        ('classify', logistic)
    ])),
    ('logistic_stem_tfidf', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', logistic)
    ])),
    ('logistic_lemmatize_count', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', count_vectorizer),
        ('classify', logistic)
    ])),
    ('logistic_lemmatize_tfidf', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', logistic)
    ])),
    ('tree_stem_count', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', count_vectorizer),
        ('classify', decision_tree)
    ])),
    ('tree_stem_tfidf', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', decision_tree)
    ])),
    ('tree_lemmatize_count', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', count_vectorizer),
        ('classify', decision_tree)
    ])),
    ('tree_lemmatize_tfidf', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', decision_tree)
    ])),
    ('bayes_stem_count', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', count_vectorizer),
        ('classify', bayes)
    ])),
    ('bayes_stem_tfidf', Pipeline([
        ('stem', stem_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', bayes)
    ])),
    ('bayes_lemmatize_count', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', count_vectorizer),
        ('classify', bayes)
    ])),
    ('bayes_lemmatize_tfidf', Pipeline([
        ('lemmatize', lemmatize_transformer),
        ('vectorize', tfidf_vectorizer),
        ('classify', bayes)
    ]))
]

In [ ]:
# Perform GridSearchCV for each pipeline
results = []

for name, pipeline in pipelines:
    # Define parameter grid based on classifier
    param_grid = {'classify__C': [0.1, 1, 10]} if 'logistic' in name else \
                 {'classify__max_depth': [None, 10, 20]} if 'tree' in name else \
                 {'classify__alpha': [0.01, 0.1, 1.0]}

    grid = GridSearchCV(pipeline, param_grid=param_grid, cv=3, verbose=1)
    
    start_time = time.time()
    grid.fit(X_train, y_train)
    elapsed_time = time.time() - start_time

    y_pred = grid.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    y_prob = grid.predict_proba(X_test)[:, 1]  # Probability of the positive class
    auc = roc_auc_score(y_test, y_prob)
    
    results.append({
        'model': name,
        'best_params': grid.best_params_,
        'best_score': grid.best_score_,
        'fit_time': elapsed_time,
        'test_score': acc_score,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'AUC': auc,
        'y pred': y_pred,
        'y_prob': y_prob
    })
    print(name, grid.best_params_, grid.best_score_, acc_score, elapsed_time)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
logistic_stem_count {'classify__C': 1} 0.88474 0.88952 612.2974989414215
Fitting 3 folds for each of 3 candidates, totalling 9 fits
logistic_stem_tfidf {'classify__C': 10} 0.8844866666666666 0.88906 678.0508050918579
Fitting 3 folds for each of 3 candidates, totalling 9 fits
logistic_lemmatize_count {'classify__C': 1} 0.8805066666666667 0.88286 213.7924029827118
Fitting 3 folds for each of 3 candidates, totalling 9 fits
logistic_lemmatize_tfidf {'classify__C': 10} 0.8802333333333334 0.88286 246.9951479434967
Fitting 3 folds for each of 3 candidates, totalling 9 fits


### Creating a DataFrame of Results

In [ ]:
# Display the results in a DataFrame
results_df = pd.DataFrame(results)
results_df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
plt.figure(figsize=(6,3))
sns.lineplot(results_df, x='model', y='fit_time')
plt.title('Compare Train Time for models')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Define the metrics to plot
metrics = ['best_score', 'test_score', 'precision', 'recall', 'f1', 'AUC']
plt.figure(figsize=(8,6))

# Plot each metric
for metric in metrics:
    sns.lineplot(data=results_df, x='model', y=metric, label=metric)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title('Compare scores for models')
plt.xticks(rotation=90)
plt.tight_layout()  # Adjust layout to make room for the legend
plt.show()

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(10, 20))  
fig.suptitle('ROC Curves for Models\n\n')  # Add a title for the entire figure

# Counter to keep track of subplot position
plot_count = 0

for index, row in results_df.iterrows():
    model = row['model']
    y_prob = row['y_prob']

    # Access current subplot based on counter
    ax = axes.flat[plot_count]  # Flattened array for easier access
    
    # Visualizing ROC Curve

    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    
    ax.plot(fpr, tpr)
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title(f"Model: {model}, AUC = {round(row['AUC'],2)}")

    plot_count += 1

# Adjust layout to prevent overlapping elements (optional)
plt.tight_layout()

plt.show()

In [ ]:
fig, axes = plt.subplots(6, 2, figsize=(10, 20))  
fig.suptitle('Confusion Matrix for Models\n\n')  # Add a title for the entire figure

# Counter to keep track of subplot position
plot_count = 0

for index, row in results_df.iterrows():
    model = row['model']
    y_pred = row['y pred'] 

    # Access current subplot based on counter
    ax = axes.flat[plot_count]  # Flattened array for easier access

    # Visualizing Confusion Matrix
    ConfusionMatrixDisplay.from_predictions(y_test, y_pred, ax=ax)
    ax.set_title(f"{model}: Confusion Matrix")

    plot_count += 1

# Adjust layout to prevent overlapping elements (optional)
plt.tight_layout()
plt.show()

### Conclusion:

- **Logistic Regression** models consistently outperform the others, especially when using stemming with count vectorization. These models offer the best balance of precision, recall, F1, and AUC, making them well-suited for text classification tasks that require accurate class identification and discrimination. Although stemming slightly improves metrics, it is more computationally expensive compared to lemmatizing.

- **Decision Trees** exhibit strong recall and F1 scores but fall short of logistic regression across all metrics.

- **Naive Bayes** models deliver competitive performance with higher computational costs and lower metrics compared to logistic regression.

Overall, the `logistic_stem_count` model stands out due to its robust performance across all key metrics. However, if computational efficiency is a concern, the `logistic_lemmatize_count` model is a viable alternative.